<a href="https://colab.research.google.com/github/weso500/IOT-Anomaly-Detection/blob/main/Single_Image_VAE_A%2BIGBT_I_GenLatent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Obtained intially from https://keras.io/examples/generative/vae/ and https://colab.research.google.com/github/keras-team/keras-io/blob/master/examples/generative/ipynb/vae.ipynb

In [1]:
pip install --upgrade keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.7 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1


In [2]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import numpy as np
import tensorflow as tf
import keras
from keras import ops
from keras import layers

In [3]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.seed_generator = keras.random.SeedGenerator(1337)

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = ops.shape(z_mean)[0]
        dim = ops.shape(z_mean)[1]
        epsilon = keras.random.normal(shape=(batch, dim), seed=self.seed_generator)
        return z_mean + ops.exp(0.5 * z_log_var) * epsilon

Use this notebook as example for obtaining images: https://colab.research.google.com/drive/1EOFDisibXb9a4A3lZ_2z4Jbhlnx0OVy6 The below takes care of obtaining images

In [4]:
latent_dim = 3

encoder_inputs = keras.Input(shape=(256, 256, 3))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 256, 256, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 128, 128, 32)   │            896 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 64, 64, 64)     │         18,496 │ conv2d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 262144)         │              0 │ conv2d_1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 128)            │     33,554,560 │ flatten[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ z_mean (Dense)            │ (None, 3)              │            387 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ z_log_var (Dense)         │ (None, 3)              │            387 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sampling (Sampling)       │ (None, 3)              │              0 │ z_mean[0][0],          │
│                           │                        │                │ z_log_var[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 33,574,726 (128.08 MB)

 Trainable params: 33,574,726 (128.08 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(64 * 64 * 64, activation="relu")(latent_inputs)
x = layers.Reshape((64, 64, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(3, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 3)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 262144)              │       1,048,576 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 64, 64, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose (Conv2DTranspose)   │ (None, 128, 128, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_1 (Conv2DTranspose) │ (None, 256, 256, 32)        │          18,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_2 (Conv2DTranspose) │ (None, 256, 256, 3)         │             867 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,104,835 (4.21 MB)

 Trainable params: 1,104,835 (4.21 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
from tensorflow.keras.layers import Layer
class KLLossLayer(Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
        kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))

        return kl_loss

class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        self.kl_loss_layer = KLLossLayer()

    def call(self, inputs):  # Add the call method
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        return reconstructed

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = ops.mean(
                ops.sum(
                    keras.losses.binary_crossentropy(data, reconstruction),
                    axis=(1, 2),
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - ops.square(z_mean) - ops.exp(z_log_var))
            kl_loss = ops.mean(ops.sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        kl_loss = self.kl_loss_layer([z_mean, z_log_var])
        return reconstructed, kl_loss, z_mean, z_log_var

    def predict(self, x):
        reconstructed, kl_loss, z_mean, z_log_var = super().predict(x)
        return reconstructed, kl_loss, z_mean, z_log_var


In [7]:
!unzip /content/Gold_Folder_Final.zip

Streaming output truncated to the last 5000 lines.
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/108Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/109Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/10Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/110Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/111Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/112Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/113Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/114Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/115Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGB

In [8]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/Gold_Folder_Final/Training_Data/Normal/Spectrograms_Normal_A+IGBT-I',
  image_size=(256, 256),
  batch_size=32,
  label_mode=None)


normalization_layer = layers.Rescaling(scale= 1./255)

normalized_ds = train_ds.map(lambda x: normalization_layer(x))

Found 500 files.


In [9]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A+IGBT-I',
  image_size=(256, 256),
  shuffle=False,
  batch_size=1,
  label_mode=None)

file_paths = test_ds.file_paths
print(file_paths)

normalization_layer = layers.Rescaling(scale= 1./255)

normalized_test_ds = test_ds.map(lambda x: normalization_layer(x))

Found 190 files.
['/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A+IGBT-I/500Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A+IGBT-I/501Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A+IGBT-I/502Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A+IGBT-I/503Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A+IGBT-I/504Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A+IGBT-I/505Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A+IGBT-I/506Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A+IGBT-I/507Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A+IGBT-I/508Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A+IGBT-I/509Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A+IGBT-I/510Norma

In [10]:
test_anomaly_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/Gold_Folder_Final/Test_Data/Anomaly/Spectrograms_Anomaly_A+IGBT-I',
  image_size=(256, 256),
  shuffle=False,
  batch_size=1,
  label_mode=None)

file_paths = test_ds.file_paths
print(file_paths)

normalization_layer = layers.Rescaling(scale= 1./255)

normalized_anomaly_test_ds = test_anomaly_ds.map(lambda x: normalization_layer(x))

Found 182 files.
['/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A+IGBT-I/500Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A+IGBT-I/501Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A+IGBT-I/502Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A+IGBT-I/503Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A+IGBT-I/504Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A+IGBT-I/505Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A+IGBT-I/506Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A+IGBT-I/507Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A+IGBT-I/508Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A+IGBT-I/509Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A+IGBT-I/510Norma

In [11]:
#(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
#mnist_digits = np.concatenate([x_train, x_test], axis=0)
#mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(normalized_ds, epochs=15, batch_size=32)

Epoch 1/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 27s 750ms/step - kl_loss: 405.9166 - loss: 44119.6094 - reconstruction_loss: 43713.6875
Epoch 2/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - kl_loss: 646.4830 - loss: 30688.6816 - reconstruction_loss: 30042.1973
Epoch 3/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step - kl_loss: 179.7385 - loss: 28827.3379 - reconstruction_loss: 28647.5996
Epoch 4/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 87ms/step - kl_loss: 102.9064 - loss: 28375.7480 - reconstruction_loss: 28272.8418
Epoch 5/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 88ms/step - kl_loss: 112.9096 - loss: 27491.8555 - reconstruction_loss: 27378.9473
Epoch 6/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step - kl_loss: 118.9287 - loss: 25637.0703 - reconstruction_loss: 25518.1406
Epoch 7/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 90ms/step - kl_loss: 70.4679 - loss: 24147.5742 - reconstruction_loss: 24077.1055
Epoch 8/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 89ms/step - kl_loss: 64.2530 - loss: 22971.1055 - reconstruction_loss: 22906.8535
Epoch 9/

In [12]:
train_ds_test_ = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/Gold_Folder_Final/Training_Data/Normal/Spectrograms_Normal_A+IGBT-I',
  image_size=(256, 256),
  batch_size=1,
  label_mode=None)


normalization_layer = layers.Rescaling(scale= 1./255)

normalized_ds_test1 = train_ds_test_.map(lambda x: normalization_layer(x))

Found 500 files.


In [13]:
pip install sewar

  Preparing metadata (setup.py) ... done
  Created wheel for sewar: filename=sewar-0.4.6-py3-none-any.whl size=11419 sha256=a4238ea62c99b0ecdd6e8e796588006331dea2e8d4fb0358611c8fed952de9e3
  Stored in directory: /root/.cache/pip/wheels/3f/af/02/9c6556ba287b62a945d737def09b8b8c35c9e1d82b9dfae84c
Successfully built sewar


In [14]:
import cv2
from urllib.request import AbstractBasicAuthHandler
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sewar.full_ref import mse, rmse, psnr, uqi, ssim, ergas, scc, rase, sam, msssim, vifp

#mse_values_normal = [] # Create an empty list to store mse values
#mse_values_anomaly = []
mse_values = []
kldiv1 = []
zmean1 = []
zmean2 = []
zmean3 = []
zlogvar1 = []
zlogvar2 = []
zlogvar3 = []

for elem in normalized_ds_test1.take(500):
  for i in range(len(elem)):
    elem_np = elem.numpy()
    exp = vae.predict(elem)
    exp_np = tf.get_static_value(exp)
    #plt.imshow(np.squeeze(elem_np))
    #plt.show()
    #plt.imshow(np.squeeze(exp_np[0]))
    #plt.show()
    #x = cv2.PSNR(elem_np, exp_np[0])
    x = mse(elem_np, exp_np[0])
    print(x)
    print(exp_np[1])
    kldiv1.append(exp_np[1])
    print(exp_np[2])
    zmean1.append(exp_np[2][0][0])
    zmean2.append(exp_np[2][0][1])
    zmean3.append(exp_np[2][0][2])
    print(exp_np[3])
    zlogvar1.append(exp_np[3][0][0])
    zlogvar2.append(exp_np[3][0][1])
    zlogvar3.append(exp_np[3][0][2])
    mse_values.append(x) # Append each mse value to the list



1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
0.00152132045991883
61.43991
[[-4.81944  -4.229932  8.558297]]
[[-3.484215  -3.8191366 -4.1442165]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
0.001533048005644185
60.547207
[[-6.184408  -6.612814   5.2881794]]
[[-4.853042  -4.9467015 -4.3254714]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
0.0015238865058313372
61.188835
[[-6.232333  -6.6628833  5.2861605]]
[[-4.887563  -4.9531636 -4.3296876]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
0.0016025776665014645
59.851242
[[-6.1505423 -6.668075   5.1073685]]
[[-4.8894386 -5.027823  -4.3810244]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
0.0015279219858628436
60.774544
[[-6.2188263 -6.648928   5.2422614]]
[[-4.875742 -4.959966 -4.322119]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
0.0014993611681960433
62.446487
[[-4.826111 -4.195285  8.696629]]
[[-3.4520931 -3.7456107 -4.100287 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
0.0015643995278505293
60.321526
[[-6.1414437 -6.551129   5.377701 ]]
[[-4.8157525 -4.9200892 -4.3242793]]
1/1 ━━━

In [15]:
import pandas as pd

dict = {'MSE': mse_values, 'KL Loss': kldiv1, 'Zmean1': zmean1, 'Zmean2': zmean2, 'Zmean3': zmean3, 'Zlogvar1': zlogvar1,'Zlogvar2': zlogvar2, 'Zlogvar3': zlogvar3 }

df = pd.DataFrame(dict)

# saving the dataframe
df.to_csv('A+IGBT-I_train.csv')

In [16]:
import cv2
from urllib.request import AbstractBasicAuthHandler
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sewar.full_ref import mse, rmse, psnr, uqi, ssim, ergas, scc, rase, sam, msssim, vifp

#mse_values_normal = [] # Create an empty list to store mse values
#mse_values_anomaly = []
mse_values = []
kldiv1 = []
zmean1 = []
zmean2 = []
zmean3 = []
zlogvar1 = []
zlogvar2 = []
zlogvar3 = []

for elem in normalized_test_ds.take(190):
  for i in range(len(elem)):
    elem_np = elem.numpy()
    exp = vae.predict(elem)
    exp_np = tf.get_static_value(exp)
    #plt.imshow(np.squeeze(elem_np))
    #plt.show()
    #plt.imshow(np.squeeze(exp_np[0]))
    #plt.show()
    #x = cv2.PSNR(elem_np, exp_np[0])
    x = mse(elem_np, exp_np[0])
    print(x)
    print(exp_np[1])
    kldiv1.append(exp_np[1])
    print(exp_np[2])
    zmean1.append(exp_np[2][0][0])
    zmean2.append(exp_np[2][0][1])
    zmean3.append(exp_np[2][0][2])
    print(exp_np[3])
    zlogvar1.append(exp_np[3][0][0])
    zlogvar2.append(exp_np[3][0][1])
    zlogvar3.append(exp_np[3][0][2])
    mse_values.append(x) # Append each mse value to the list

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
0.002582923097591707
75.35568
[[-7.627783  -8.77498    1.8393409]]
[[-5.278047  -5.8459735 -3.9944236]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
0.002570089494031588
76.22662
[[-7.680504  -8.834628   1.7911654]]
[[-5.3042564 -5.883028  -3.9906657]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
0.0023962080343205646
73.40884
[[-7.503759  -8.630782   1.9712548]]
[[-5.257941  -5.8058276 -4.0455375]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
0.002450868910226721
76.05199
[[-7.661171  -8.849439   1.6631625]]
[[-5.3322115 -5.935516  -4.038935 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
0.0024320445852103593
74.36136
[[-7.577099  -8.626813   2.1999478]]
[[-5.239801  -5.7696652 -4.0126357]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
0.002355843250636804
75.68459
[[-7.647582  -8.767195   1.9577849]]
[[-5.2933517 -5.855801  -4.0119095]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
0.0024016371996646215
76.82751
[[-7.6982155 -8.868374   1.8498554]]
[[-5.3561144 -5.914204  -4.0268893]]
1/1

In [17]:
import pandas as pd

dict = {'MSE': mse_values, 'KL Loss': kldiv1, 'Zmean1': zmean1, 'Zmean2': zmean2, 'Zmean3': zmean3, 'Zlogvar1': zlogvar1,'Zlogvar2': zlogvar2, 'Zlogvar3': zlogvar3 }

df2 = pd.DataFrame(dict)

# saving the dataframe
df2.to_csv('A+IGBT-I_testnorm.csv')

In [18]:
import cv2
from urllib.request import AbstractBasicAuthHandler
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sewar.full_ref import mse, rmse, psnr, uqi, ssim, ergas, scc, rase, sam, msssim, vifp

#mse_values_normal = [] # Create an empty list to store mse values
#mse_values_anomaly = []
mse_values = []
kldiv1 = []
zmean1 = []
zmean2 = []
zmean3 = []
zlogvar1 = []
zlogvar2 = []
zlogvar3 = []

for elem in normalized_anomaly_test_ds.take(190):
  for i in range(len(elem)):
    elem_np = elem.numpy()
    exp = vae.predict(elem)
    exp_np = tf.get_static_value(exp)
    #plt.imshow(np.squeeze(elem_np))
    #plt.show()
    #plt.imshow(np.squeeze(exp_np[0]))
    #plt.show()
    #x = cv2.PSNR(elem_np, exp_np[0])
    x = mse(elem_np, exp_np[0])
    print(x)
    print(exp_np[1])
    kldiv1.append(exp_np[1])
    print(exp_np[2])
    zmean1.append(exp_np[2][0][0])
    zmean2.append(exp_np[2][0][1])
    zmean3.append(exp_np[2][0][2])
    print(exp_np[3])
    zlogvar1.append(exp_np[3][0][0])
    zlogvar2.append(exp_np[3][0][1])
    zlogvar3.append(exp_np[3][0][2])
    mse_values.append(x) # Append each mse value to the list

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
0.0289808199295825
31.792744
[[-3.4475079 -4.0881596  4.977011 ]]
[[-3.1887658 -5.2006426 -4.771878 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
0.01422105071093143
43.37899
[[-4.392518  -4.3288693  6.322343 ]]
[[-3.3180723 -4.318045  -4.0495496]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
0.03425697888691275
34.275513
[[-4.110252  -4.416774   4.8567734]]
[[-3.2477412 -4.335866  -3.905012 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
0.001622531868249482
61.99562
[[-4.8865027 -4.2868934  8.55844  ]]
[[-3.5025752 -3.8017302 -4.1159034]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
0.06304077059942319
31.815266
[[-4.409445  -4.587423   4.0014963]]
[[-3.3328316 -3.6069582 -3.0824428]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
0.02684024831325174
34.89055
[[-4.330735  -4.9533353  4.024384 ]]
[[-3.4940388 -5.2961006 -4.4575124]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
0.001540264742013497
62.597824
[[-4.794764 -4.136296  8.760836]]
[[-3.4226847 -3.7430074 -4.106233 ]]
1/1 ━━━━━━━

In [19]:
import pandas as pd

dict = {'MSE': mse_values, 'KL Loss': kldiv1, 'Zmean1': zmean1, 'Zmean2': zmean2, 'Zmean3': zmean3, 'Zlogvar1': zlogvar1,'Zlogvar2': zlogvar2, 'Zlogvar3': zlogvar3 }

df3 = pd.DataFrame(dict)

# saving the dataframe
df3.to_csv('A+IGBT-I_testanom.csv')